In [1]:
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
import warnings
warnings.filterwarnings('ignore')

import command
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,'

import random
import copy
import math
import shutil
import wandb
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from glob import glob

import segmentation_models_pytorch as smp
import timm

from sklearn.model_selection import *
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch

import albumentations as A
from albumentations.pytorch import ToTensorV2

from transformers import get_cosine_schedule_with_warmup

In [2]:
class CFG:
    FOLD = 0
    FULLDATA = False
    
    image_size = [480, 480]
    seed = 1123
    
    device = torch.device('cuda')
    v = -1
    
    n_folds = 5
    n_epochs = 10
    
    model_name = "tf_efficientnet_b3_ns"
    
    train_batch_size = 16
    valid_batch_size = 16
    acc_steps = 1
    
    wd = 1e-6
    n_warmup_steps = 0
    upscale_steps = 1.1
    validate_every = 1
    
    epoch = 0
    global_step = 0 
    literal_step = 0
    
    autocast = True
    

    workers = 0
    
if CFG.FULLDATA:
    CFG.seed = CFG.FOLD

In [3]:
#for batch in tqdm(valid_loader): break

In [4]:
'''images = batch['images']
image = images[0]
flip_images = torch.flip(images, (2,))
plt.imshow(flip_images[0].permute(1, 2, 0))'''

"images = batch['images']\nimage = images[0]\nflip_images = torch.flip(images, (2,))\nplt.imshow(flip_images[0].permute(1, 2, 0))"

In [5]:
data = pd.read_csv("/mnt/md0/wns_triangle/test/test.csv")
data

,image_id,filename
0,8080,8080.jpg
1,8081,8081.jpg
2,8082,8082.jpg
3,8083,8083.jpg
4,8084,8084.jpg
...,...,...
3457,11537,11537.jpg
3458,11538,11538.jpg
3459,11539,11539.jpg
3460,11540,11540.jpg


In [6]:
class WNSDataset(Dataset):
    def __init__(self, data, transforms=None):
        self.data = data
        self. transforms = transforms
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, i):
        row = self.data.iloc[i]
        file = row["filename"].replace(".jpg", ".npy")
        ids = row["image_id"]
        
        image = np.load(f"/mnt/md0/wns_triangle/test/images/{file}")
        
        if self.transforms:
            transformed = self.transforms(image=image)
            
            image = transformed["image"]
            if image.dtype==torch.uint8: image = image.float() / 255
        
        return {"images": image,
               "ids": ids}
            

In [7]:
def get_loaders(test_augs=None):
    valid_data = data
    
    if test_augs:
        valid_augs = test_augs
    else:
        valid_augs = A.Compose([
            A.Resize(CFG.image_size[0], CFG.image_size[1]),
            #A.Normalize(max_pixel_value=255),
            ToTensorV2(),
        ])
    
    valid_dataset = WNSDataset(valid_data, valid_augs)
    
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.valid_batch_size, shuffle=False, num_workers=CFG.workers, pin_memory=False)
    
    return valid_loader

valid_loader = get_loaders()

for d in tqdm(valid_loader):
     break


  0%|                                                   | 0/217 [00:00<?, ?it/s]


In [8]:
class Model_conv(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.backbone = timm.create_model(CFG.model_name, pretrained=True, num_classes=0, global_pool='')
        
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
        self.feats = self.backbone.num_features
        
        self.head = nn.Linear(self.feats, 1)
    
    def forward(self, inp):
        features = self.backbone(inp)
        
        features = self.avgpool(features).flatten(1, 3)
        
        logits = self.head(features)
        
        return logits
    
class Model_trans(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.backbone = timm.create_model(CFG.model_name, pretrained=True, num_classes=0, global_pool='')
        
        self.feats = self.backbone.num_features
        
        self.head = nn.Linear(self.feats, 1)
    
    def forward(self, inp):
        features = self.backbone(inp)
        
        features = features.mean(1)
        
        logits = self.head(features)
        
        return logits

In [9]:
#CFG.model_name = 'vit_base_patch16_224'
#Model_vit()

In [10]:
def valid_one_epoch(path, loader, running_dist=False, debug=False):
    model = Model()
    st = torch.load(path, map_location=f"cpu")
    model.eval()
    model.cuda()
    model.load_state_dict(st, strict=False)
    
    if is_main_process(): bar = tqdm(loader, bar_format='{n_fmt}/{total_fmt} {elapsed}<{remaining} {postfix}')
    else: bar = loader
    
    running_loss = 0.
    
    OUTPUTS = []
    TARGETS = []
    MASKS_TARGETS = []
    IDS = []
    
    for step, data in enumerate(bar):
        with torch.no_grad():
            images = data['images'].cuda()
            ids = data['ids']
            
            with torch.cuda.amp.autocast(enabled=CFG.autocast):
                logits = model(images)
                logits2 = model(torch.flip(images, (3,)))
                
                #logits = (logits + logits2) / 2
                
            outputs = logits.float().sigmoid().detach().cpu()#.numpy()            
            outputs2 = logits2.float().sigmoid().detach().cpu()#.numpy()
            outputs = (outputs + outputs2) / 2

            OUTPUTS.extend(outputs)
            IDS.extend(ids)
    
    OUTPUTS = np.stack(OUTPUTS)
    IDS = np.stack(IDS)
    
    return OUTPUTS, IDS

In [11]:
def is_main_process(): return 1

In [12]:
OUTPUTS1 = []

In [13]:
#'''
Model = Model_conv
CFG.model_name = 'tf_efficientnetv2_xl.in21k_ft_in1k'
CFG.image_size = [512, 512]
test_augs = A.Compose([
    A.Resize(CFG.image_size[0], CFG.image_size[1]),
    #A.Normalize(max_pixel_value=255),
    ToTensorV2(),
])
valid_loader = get_loaders(test_augs)
OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v61/0.pth', valid_loader)
OUTPUTS1.append(OUTPUTS)
#'''

217/217 01:58<00:00 


In [14]:
#'''
for F in [3]:
    Model = Model_conv
    CFG.model_name = 'tf_efficientnetv2_xl.in21k_ft_in1k'
    CFG.image_size = [512, 512]
    test_augs = A.Compose([
        A.Resize(CFG.image_size[0], CFG.image_size[1]),
        #A.Normalize(max_pixel_value=255),
        ToTensorV2(),
    ])
    valid_loader = get_loaders(test_augs)
    OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v69/{F}.pth', valid_loader)
    OUTPUTS1.append(OUTPUTS)
#'''

217/217 01:53<00:00 


In [15]:
#'''
Model = Model_conv
CFG.model_name = 'tf_efficientnetv2_l.in21k_ft_in1k'
CFG.image_size = [480, 480]
test_augs = A.Compose([
    A.Resize(CFG.image_size[0], CFG.image_size[1]),
    #A.Normalize(max_pixel_value=255),
    ToTensorV2(),
])
valid_loader = get_loaders(test_augs)
OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v59/0.pth', valid_loader)
OUTPUTS1.append(OUTPUTS)
#'''

217/217 01:21<00:00 


In [16]:
#'''
for F in [3]:
    Model = Model_conv
    CFG.model_name = 'tf_efficientnetv2_l.in21k_ft_in1k'
    CFG.image_size = [480, 480]
    test_augs = A.Compose([
        A.Resize(CFG.image_size[0], CFG.image_size[1]),
        #A.Normalize(max_pixel_value=255),
        ToTensorV2(),
    ])
    valid_loader = get_loaders(test_augs)
    OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v70/{F}.pth', valid_loader)
    OUTPUTS1.append(OUTPUTS)
#'''

217/217 01:21<00:00 


In [17]:
#'''
for F in [0]:
    Model = Model_conv
    CFG.model_name = 'tf_efficientnetv2_l.in21k_ft_in1k'
    CFG.image_size = [480, 480]
    test_augs = A.Compose([
        A.Resize(CFG.image_size[0], CFG.image_size[1]),
        #A.Normalize(max_pixel_value=255),
        ToTensorV2(),
    ])
    valid_loader = get_loaders(test_augs)
    OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v82/{F}.pth', valid_loader)
    OUTPUTS1.append(OUTPUTS)
#'''

217/217 01:20<00:00 


In [18]:
#'''
for F in [3]:
    Model = Model_conv
    CFG.model_name = 'tf_efficientnetv2_l.in21k_ft_in1k'
    CFG.image_size = [480, 480]
    test_augs = A.Compose([
        A.Resize(CFG.image_size[0], CFG.image_size[1]),
        #A.Normalize(max_pixel_value=255),
        ToTensorV2(),
    ])
    valid_loader = get_loaders(test_augs)
    OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v86/{F}.pth', valid_loader)
    OUTPUTS1.append(OUTPUTS)
#'''

217/217 01:20<00:00 


In [30]:
#'''
Model = Model_conv
CFG.model_name = 'tf_efficientnetv2_l.in21k_ft_in1k'
CFG.image_size = [640, 640]
test_augs = A.Compose([
    A.Resize(CFG.image_size[0], CFG.image_size[1]),
    #A.Normalize(max_pixel_value=255),
    ToTensorV2(),
])
valid_loader = get_loaders(test_augs)
OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v129/0.pth', valid_loader)
OUTPUTS1.append(OUTPUTS)
#'''

217/217 02:08<00:00 


In [31]:
#'''
Model = Model_conv
CFG.model_name = 'tf_efficientnetv2_l.in21k_ft_in1k'
CFG.image_size = [640, 640]
test_augs = A.Compose([
    A.Resize(CFG.image_size[0], CFG.image_size[1]),
    #A.Normalize(max_pixel_value=255),
    ToTensorV2(),
])
valid_loader = get_loaders(test_augs)
OUTPUTS, IDS = valid_one_epoch(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/{CFG.model_name}_v129/3.pth', valid_loader)
OUTPUTS1.append(OUTPUTS)
#'''

217/217 02:08<00:00 


In [32]:
def voting_in_confidence(outputs, q_in):
    votes = []
    for output in outputs:
        thresh = np.quantile(output, 1-q_in)
        votes.append((output>thresh).astype(np.int32))
    
    confs = outputs.mean(0)
    votes = np.array(votes)
    votes = votes.sum(0)
    
    conf_vote = votes + confs
    
    return conf_vote

In [33]:
OUTPUTS_1 = np.stack(OUTPUTS1)
OUTPUTS_1.shape

(8, 3462, 1)

In [34]:
FINAL_OUTPUTS = voting_in_confidence(OUTPUTS_1, 0.0600)
FINAL_OUTPUTS.shape

(3462, 1)

In [35]:
threshold = np.quantile(FINAL_OUTPUTS, 1-0.0575)
threshold

4.151550909355281

In [36]:
final_outputs = (FINAL_OUTPUTS>threshold).astype(np.int32)

In [37]:
sub = pd.DataFrame({'image_id': IDS, 'label': final_outputs[:, 0]})

In [38]:
sub

,image_id,label
0,8080,0
1,8081,0
2,8082,0
3,8083,0
4,8084,1
...,...,...
3457,11537,0
3458,11538,0
3459,11539,0
3460,11540,0


In [39]:
sub.label.value_counts()

label
0    3262
1     200
Name: count, dtype: int64

In [40]:
sub.to_csv(f'/mnt/md0/wns_triangle/AAA_SEG/TRY1_SEG/submits/v70_v69_v89_v129_f0_f3_vcq600_q575.csv', index=False)

In [ ]:
#pd.read_csv('/mnt/md0/wns_triangle/sample_submission_R0y2qV4.csv')